## Recreating the following for classification problem
https://www.tensorflow.org/text/tutorials/transformer

Need tensorflow-text for using the BERT model tokenizer

In [1]:
# Install the nightly version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install -q -U tensorflow-text tensorflow

     |████████████████████████████████| 5.9 MB 9.4 MB/s 
     |████████████████████████████████| 578.0 MB 17 kB/s 
     |████████████████████████████████| 438 kB 77.5 MB/s 
     |████████████████████████████████| 1.7 MB 54.7 MB/s 
     |████████████████████████████████| 5.9 MB 47.7 MB/s 


Import the necessary modules:

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_text
from sklearn.utils import shuffle 

Doing necessary data preprocessing

In [3]:
os.listdir('.')

['.config', 'twitter_training.csv', 'twitter_validation.csv', 'sample_data']

In [4]:
training_data_loc = "./twitter_training.csv"
val_data_loc = "./twitter_validation.csv"

In [5]:
df = pd.read_csv(training_data_loc)
df

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [6]:
df = df[df.columns[2:]]

In [7]:
df = df.rename(columns={"Positive":"Sentiment", "im getting on borderlands and i will murder you all ,":"sentence"})

In [8]:
df['Sentiment'].value_counts()

Negative      22542
Positive      20831
Neutral       18318
Irrelevant    12990
Name: Sentiment, dtype: int64

In [9]:
df = df[df.apply(lambda x: (x["Sentiment"] == "Positive") or (x["Sentiment"] == "Negative"), axis=1)]

In [10]:
df['Sentiment'].value_counts()

Negative    22542
Positive    20831
Name: Sentiment, dtype: int64

In [11]:
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == "Positive" else 0)

In [12]:
df = shuffle(df)

df

,Sentiment,sentence
74310,1,Congratulations to the NVIDIA NeMo team on Rel...
70186,1,I was telling the sub let that the sky isn’t e...
72327,0,A @GhostRecon when again are really you finall...
63658,0,@EAMaddenNFL whTs the problem now with EA? Why...
64783,0,@ EAMaddenNFL why y'all hate franchise players...
...,...,...
41078,0,"Sorry, my bad .. www.playstation.com/"
13791,0,with RhandlerR how does ur cover athletes jump...
15354,0,Wtf.
50494,0,Pissing people off within FIFA and on twitter.


In [13]:
df.isna().value_counts()

Sentiment  sentence
False      False       43012
           True          361
dtype: int64

In [14]:
df.dropna(inplace=True) # dropping empty sentences

In [15]:
df

,Sentiment,sentence
74310,1,Congratulations to the NVIDIA NeMo team on Rel...
70186,1,I was telling the sub let that the sky isn’t e...
72327,0,A @GhostRecon when again are really you finall...
63658,0,@EAMaddenNFL whTs the problem now with EA? Why...
64783,0,@ EAMaddenNFL why y'all hate franchise players...
...,...,...
41078,0,"Sorry, my bad .. www.playstation.com/"
13791,0,with RhandlerR how does ur cover athletes jump...
15354,0,Wtf.
50494,0,Pissing people off within FIFA and on twitter.


In [16]:
labels = df['Sentiment']
features = df['sentence']

## Tokenizer
using subword tokenizer implementation from Bert model

In [17]:
model_name = 'ted_hrlr_translate_pt_en_converter'
tf.keras.utils.get_file(
    f'{model_name}.zip',
    f'https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip',
    cache_dir='.', cache_subdir='', extract=True
)

184801/184801 [==============================] - 0s 0us/step


'./ted_hrlr_translate_pt_en_converter.zip'

In [19]:
tokenizers = tf.saved_model.load(model_name)

In [20]:
[item for item in dir(tokenizers) if not item.startswith('_')]

['en',
 'graph_debug_info',
 'pt',
 'signatures',
 'tensorflow_git_version',
 'tensorflow_version']

The `tf.saved_model` contains two text tokenizers, one for English and one for Portuguese. We just need the English one

In [21]:
[item for item in dir(tokenizers.en) if not item.startswith('_')]

['detokenize',
 'get_reserved_tokens',
 'get_vocab_path',
 'get_vocab_size',
 'lookup',
 'tokenize',
 'tokenizer',
 'vocab']

In [22]:
sample = features.sample(n=5)
sample

12107                                   This bothers me.  
2544                    Not overly confident on this issue
41837    Update: I did not do this. . I played battlefi...
17833    oooooh shit i think my motherboard is already ...
57060    I really need to start playing @ Rainbow6Game,...
Name: sentence, dtype: object

In [23]:
text_sample = sample.iloc[0]
print(text_sample)

This bothers me.  


In [24]:
encoded = tokenizers.en.tokenize(sample)
# print(encoded)
print('Padded and encoded embeddings:')
for row in encoded.to_list():
  print(row)

Padded and encoded embeddings:
[2, 81, 4358, 88, 114, 15, 3]
[2, 98, 187, 188, 2803, 92, 81, 894, 3]
[2, 130, 5933, 27, 45, 149, 98, 91, 81, 15, 15, 45, 1868, 2312, 6438, 21, 501, 15, 148, 84, 37, 2395, 15, 3]
[2, 51, 3178, 3178, 459, 3529, 45, 133, 99, 519, 2491, 80, 418, 3456, 4312, 3746, 836, 3]
[2, 45, 140, 181, 73, 260, 1488, 31, 1470, 6401, 1163, 5140, 1572, 13, 3213, 45, 153, 89, 121, 1411, 417, 171, 31, 2159, 773, 6464, 587, 6422, 2199, 230, 85, 59, 1325, 388, 388, 388, 3]


The padded 2 and 3 stands for [START] and [END]

In [28]:
encoded[:, :128]

<tf.RaggedTensor [[2, 81, 4358, 88, 114, 15, 3], [2, 98, 187, 188, 2803, 92, 81, 894, 3],
 [2, 130, 5933, 27, 45, 149, 98, 91, 81, 15, 15, 45, 1868, 2312, 6438, 21,
  501, 15, 148, 84, 37, 2395, 15, 3]                                      ,
 [2, 51, 3178, 3178, 459, 3529, 45, 133, 99, 519, 2491, 80, 418, 3456, 4312,
  3746, 836, 3]                                                             ,
 [2, 45, 140, 181, 73, 260, 1488, 31, 1470, 6401, 1163, 5140, 1572, 13,
  3213, 45, 153, 89, 121, 1411, 417, 171, 31, 2159, 773, 6464, 587, 6422,
  2199, 230, 85, 59, 1325, 388, 388, 388, 3]                             ]>

In [29]:
round_trip = tokenizers.en.detokenize(encoded[:, :128])

print('Original text:')
for line in round_trip.numpy():
  print(line.decode('utf-8'))

Original text:
this bothers me .
not overly confident on this issue
update : i did not do this . . i played battlefield 4 instead . which was a mistake .
oooooh shit i think my motherboard is already compatible
i really need to start playing @ rainbow6game , definitely i will have more useful experience than @ destinythegame for wwiii


## Set up a data pipeline with `tf.data`

The following function takes batches of text as input, and converts them to a format suitable for training. 

1. It tokenizes them into ragged batches.
2. It trims each to be no longer than `MAX_TOKENS`.
3. It splits the output (English) tokens into inputs and labels. THese shifted by one step so that the `label` at each location is the id of the next token.
4. It converts the `RaggedTensor`s to padded dense `Tensor`s.
5. It returns an `(inputs, labels)` pair.

NOTE: We will bypass this method in the batch making area and utilize our own method since we are not using `tf.data`

# **Tokenizer**
Cannot prepare batches as required. I don't know how to fix it.

In [31]:
MAX_TOKENS=128
def prepare_batch(en): # not actually preparing batches
    en = tokenizers.en.tokenize(en)
    en = en[:, :(MAX_TOKENS+1)] # pads with `.` if the sentence is less than 128 words and strips if more than 128 words
    # print(en)
    # en_inputs = en[:, :-1].to_tensor()
    return en

In [32]:
tokenized_sentences = prepare_batch(features)

In [33]:
list_tokenized = tokenized_sentences.to_list()

In [ ]:
list_tokenized

In [35]:
len(list_tokenized), len(labels)

(43012, 43012)

In [36]:
for i in range(len(list_tokenized)-43000):
  print(list_tokenized[i], labels.iloc[i])

[2, 3455, 5162, 1305, 271, 2714, 88, 73, 71, 50, 2204, 1496, 858, 50, 4096, 376, 755, 92, 1903, 3599, 18, 15, 17, 15, 17, 4, 140, 1401, 73, 125, 50, 4096, 376, 2517, 44, 240, 178, 1305, 73, 184, 492, 74, 71, 5244, 3133, 429, 74, 659, 507, 4, 27, 11, 3] 1
[2, 45, 84, 800, 71, 2954, 186, 75, 71, 1493, 80, 269, 68, 56, 3337, 48, 2564, 173, 235, 45, 1471, 15, 1401, 85, 45, 4864, 2430, 3826, 15, 3] 1
[2, 37, 31, 43, 5502, 773, 966, 6407, 117, 273, 86, 140, 79, 605, 6157, 71, 2312, 2537, 88, 30, 72, 166, 1102, 224, 114, 79, 68, 105, 112, 2928, 71, 5695, 2611, 1409, 484, 72, 71, 5260, 60, 601, 444, 1409, 484, 15, 45, 302, 698, 75, 1243, 82, 75, 79, 94, 283, 266, 73, 15, 285, 1199, 71, 444, 217, 601, 85, 163, 74, 134, 136, 86, 320, 102, 71, 859, 49, 3895, 15, 3] 0
[2, 31, 41, 2489, 5934, 269, 1192, 392, 59, 459, 803, 71, 249, 110, 93, 41, 303, 30, 192, 71, 3] 0
[2, 31, 41, 2489, 5934, 269, 1192, 392, 192, 61, 9, 101, 1988, 42, 1305, 269, 1173, 2709, 6258, 82, 170, 90, 78, 91, 93, 61, 9, 101, 3

## Copied code 
we need to change it up a bit so we can use same definitions for our own classification problem

### Define the Embedding and Positional encoding

In [37]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)
  
  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

  return tf.cast(pos_encoding, dtype=tf.float32)

In [39]:
temp_encoding = positional_encoding(length=2048, depth=512)
temp_encoding

<tf.Tensor: shape=(2048, 512), dtype=float32, numpy=
array([[ 0.        ,  0.        ,  0.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 0.84147096,  0.8218562 ,  0.8019618 , ...,  1.        ,
         1.        ,  1.        ],
       [ 0.9092974 ,  0.9364147 ,  0.95814437, ...,  1.        ,
         1.        ,  1.        ],
       ...,
       [ 0.17589758, -0.18608274, -0.7070546 , ...,  0.9741639 ,
         0.97595036,  0.97761387],
       [-0.7333133 ,  0.7014913 ,  0.1447375 , ...,  0.9741387 ,
         0.97592694,  0.97759205],
       [-0.9683193 ,  0.98535496,  0.8799798 , ...,  0.9741135 ,
         0.9759035 ,  0.9775702 ]], dtype=float32)>

In [40]:
temp_encoding[tf.newaxis, :128, :].shape

TensorShape([1, 128, 512])

In [38]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x # (batch_size, length-of-signal, 512)


In [42]:
x = np.zeros((400, 128, 512))
y = np.zeros((1, 128, 512))

In [51]:
x = np.random.rand(4, 3, 2)
y = np.random.rand(1, 3, 2)


In [53]:
x, y

(array([[[0.04402398, 0.42241895],
         [0.1701536 , 0.54974911],
         [0.93333992, 0.58173488]],
 
        [[0.97077209, 0.93875453],
         [0.14166638, 0.64795212],
         [0.27086614, 0.61109702]],
 
        [[0.34179167, 0.2351355 ],
         [0.22930811, 0.99207833],
         [0.01065292, 0.50036966]],
 
        [[0.60377925, 0.67056935],
         [0.2915253 , 0.42764839],
         [0.8933719 , 0.99887264]]]), array([[[0.28158527, 0.27949122],
         [0.52562244, 0.12711617],
         [0.49835702, 0.21595887]]]))

In [52]:
x+y

array([[[0.32560925, 0.70191017],
        [0.69577604, 0.67686528],
        [1.43169693, 0.79769375]],

       [[1.25235736, 1.21824575],
        [0.66728882, 0.77506829],
        [0.76922316, 0.82705589]],

       [[0.62337693, 0.51462672],
        [0.75493055, 1.1191945 ],
        [0.50900993, 0.71632853]],

       [[0.88536451, 0.95006057],
        [0.81714774, 0.55476455],
        [1.39172891, 1.21483151]]])

In [ ]:
tf.cast(512, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=512.0>

In [ ]:
tokenizers.en.get_vocab_size()

<tf.Tensor: shape=(), dtype=int32, numpy=7010>

In [ ]:
temp_layer = tf.keras.layers.Embedding(7010, 512, mask_zero=True)
# help(temp_layer)

In [ ]:
temp_model = tf.keras.Sequential()
temp_layer = tf.keras.layers.Embedding(1000, 64, input_length=10, mask_zero=True)
temp_model.add(temp_layer)
# The model will take as input an integer matrix of size (batch,
# input_length), and the largest integer (i.e. word index) in the input
# should be no larger than 999 (vocabulary size).
# Now model.output_shape is (None, 10, 64), where `None` is the batch
# dimension.
input_array = np.random.randint(1000, size=(1, 10))
temp_model.compile('rmsprop', 'mse')
output_array = temp_model.predict(input_array)
# print(output_array)
print(output_array.shape)
#(1, 10, 64)

1/1 [==============================] - 0s 84ms/step
(1, 10, 64)


In [ ]:
# print(temp_layer.compute_mask(input_array, mask_zero=True)) # would not work as this is just for forwarding the compute_mask

In [54]:
embed_en = PositionalEmbedding(vocab_size=tokenizers.en.get_vocab_size(), d_model=512)
embed_en

In [55]:
encoded.to_tensor()

<tf.Tensor: shape=(5, 37), dtype=int64, numpy=
array([[   2,   81, 4358,   88,  114,   15,    3,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   2,   98,  187,  188, 2803,   92,   81,  894,    3,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   2,  130, 5933,   27,   45,  149,   98,   91,   81,   15,   15,
          45, 1868, 2312, 6438,   21,  501,   15,  148,   84,   37, 2395,
          15,    3,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   2,   51, 3178, 3178,  459, 3529,   45,  133,   99,  519, 2491,
          80,  418, 3456, 4312, 3746,  836,    3,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   

In [56]:
# This is where the error pops up. I assume it is because of the way we are preparing batches.

# embed_en(encoded)
embed_en(encoded.to_tensor()).shape

TensorShape([5, 37, 512])

### Define the feed forward network

Define a function for the point-wise feed-forward network that you'll use later.

The network consists of two linear layers (`tf.keras.layers.Dense`) with a ReLU activation in-between:

In [57]:
def point_wise_feed_forward_network(
  d_model, # Input/output dimensionality.
  dff # Inner-layer dimensionality.
  ):

  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # Shape `(batch_size, seq_len, dff)`.
      tf.keras.layers.Dense(d_model)  # Shape `(batch_size, seq_len, d_model)`.
  ])

### Define the encoder layer

In [58]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               dropout_rate=0.1
               ):
    super().__init__()


    # Multi-head self-attention.
    self.mha = tf.keras.layers.MultiHeadAttention(
        num_heads=num_attention_heads,
        key_dim=d_model, # Size of each attention head for query Q and key K.
        dropout=dropout_rate,
        )
    # Point-wise feed-forward network.
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    # Layer normalization.
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    # Dropout for the point-wise feed-forward network.
    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x, training, mask):

    # A boolean mask.
    if mask is not None:
      mask1 = mask[:, :, None]
      mask2 = mask[:, None, :]
      attention_mask = mask1 & mask2
    else:
      attention_mask = None

    # Multi-head self-attention output (`tf.keras.layers.MultiHeadAttention `).
    attn_output = self.mha(
        query=x,  # Query Q tensor.
        value=x,  # Value V tensor.
        key=x,  # Key K tensor.
        attention_mask=attention_mask, # A boolean mask that prevents attention to certain positions.
        training=training, # A boolean indicating whether the layer should behave in training mode.
        )

    # Multi-head self-attention output after layer normalization and a residual/skip connection.
    out1 = self.layernorm1(x + attn_output)  # Shape `(batch_size, input_seq_len, d_model)`

    # Point-wise feed-forward network output.
    ffn_output = self.ffn(out1)  # Shape `(batch_size, input_seq_len, d_model)`
    ffn_output = self.dropout1(ffn_output, training=training)
    # Point-wise feed-forward network output after layer normalization and a residual skip connection.
    out2 = self.layernorm2(out1 + ffn_output)  # Shape `(batch_size, input_seq_len, d_model)`.

    return out2

In [59]:
sample_encoder_layer = EncoderLayer(d_model=512, num_attention_heads=8, dff=2048)

sample_encoder_layer_output = sample_encoder_layer(
    tf.random.uniform((2, 3, 512)), training=False, mask=None)

# Print the shape.
print(sample_encoder_layer_output.shape)  # Shape `(batch_size, input_seq_len, d_model)`.

(2, 3, 512)


### Define the encoder

In [69]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self,
               *,
               num_layers,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               input_vocab_size, # Input (Portuguese) vocabulary size.
               dropout_rate=0.1
               ):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    # Embeddings + Positional encoding
    self.pos_embedding = PositionalEmbedding(input_vocab_size, d_model)

    # Encoder layers.
    self.enc_layers = [
        EncoderLayer(
          d_model=d_model,
          num_attention_heads=num_attention_heads,
          dff=dff,
          dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    # Dropout.
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  # Masking.
  def compute_mask(self, x, previous_mask=None):
    return self.pos_embedding.compute_mask(x, previous_mask)

  def call(self, x, training):

    seq_len = tf.shape(x)[1]

    # Sum up embeddings and positional encoding.
    mask = self.compute_mask(x)
    x = self.pos_embedding(x)  # Shape `(batch_size, input_seq_len, d_model)`.
    # Add dropout.
    x = self.dropout(x, training=training)

    # N encoder layers.
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    # experimental thing, may not work
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    return x  # Shape `(batch_size, input_seq_len, d_model)`.

In [70]:
# Instantiate the encoder.
sample_encoder = Encoder(
    num_layers=8,
    d_model=512,
    num_attention_heads=8,
    dff=2048,
    input_vocab_size=7010)

# Set the test input.
sample_encoder_output = sample_encoder(encoded.to_tensor(),
                                       training=False)

# Print the shape.
print(encoded.to_tensor().shape)
print(sample_encoder_output.shape)  # Shape `(batch_size, input_seq_len, d_model)`.

(5, 37)
(5, 1)


In [68]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(128)),
    sample_encoder
])

In [66]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  87614464  
                                                                 
Total params: 87,614,464
Trainable params: 87,614,464
Non-trainable params: 0
_________________________________________________________________


In [ ]:
[item for item in dir(model) if not item.startswith('_')]

['activity_regularizer',
 'add',
 'add_loss',
 'add_metric',
 'add_update',
 'add_variable',
 'add_weight',
 'build',
 'built',
 'call',
 'compile',
 'compiled_loss',
 'compiled_metrics',
 'compute_dtype',
 'compute_loss',
 'compute_mask',
 'compute_metrics',
 'compute_output_shape',
 'compute_output_signature',
 'count_params',
 'distribute_strategy',
 'dtype',
 'dtype_policy',
 'dynamic',
 'evaluate',
 'evaluate_generator',
 'finalize_state',
 'fit',
 'fit_generator',
 'from_config',
 'get_config',
 'get_input_at',
 'get_input_mask_at',
 'get_input_shape_at',
 'get_layer',
 'get_output_at',
 'get_output_mask_at',
 'get_output_shape_at',
 'get_weight_paths',
 'get_weights',
 'history',
 'inbound_nodes',
 'input',
 'input_mask',
 'input_names',
 'input_shape',
 'input_spec',
 'inputs',
 'layers',
 'load_weights',
 'losses',
 'make_predict_function',
 'make_test_function',
 'make_train_function',
 'metrics',
 'metrics_names',
 'name',
 'name_scope',
 'non_trainable_variables',
 'non_tra

In [ ]:
model.input_shape

(None, 128)

In [ ]:
tokenized_sentences.to_tensor().shape # iske batches bnane hain...

TensorShape([43012, 129])